In [16]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate
pd.options.display.float_format = '{:,.1f}'.format

In [33]:
q='''select name, count(*) as total from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        and language in ('C','C++')
        group by name
        order by name'''
ta = pd.DataFrame(sql.execute(q)) 
q='''select name, count(*) as memory from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        and me.memory =1 
        and language in ('C','C++')
        group by name
        order by name'''
tma = pd.DataFrame(sql.execute(q)) 
tma['memory (%)']= round((tma.memory / ta.total) * 100,2)
q='''select name, count(*) as non_memory from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        and me.memory is null
        and language in ('C','C++')
        group by name
        order by name'''
tnma = pd.DataFrame(sql.execute(q)) 
tnma['non_memory (%)']= round((tnma.non_memory / ta.total) * 100,2)
df= reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])   
df = df.sort_values('total',ascending=False)

In [18]:
df['memory (%)'].median()

35.445

In [34]:
df['memory']= df['memory'].apply(lambda x: '{:,}'.format(x))
df['memory']= df['memory'].astype(str)+' ('+df['memory (%)'].astype(str)+'%)'
df=df.drop(columns=['non_memory', 'non_memory (%)', 'memory (%)'])
df=df.rename(columns={'name':'Project','total':'Total Alerts', 'memory':'Memory Alerts'})
df['Total Alerts']=df['Total Alerts'].apply(lambda x: '{:,}'.format(x))
print(tabulate(df, tablefmt='pipe',showindex=False, headers='keys'))

| Project               | Total Alerts   | Memory Alerts   |
|:----------------------|:---------------|:----------------|
| Linux                 | 19,514         | 7,903 (40.5%)   |
| Firefox               | 12,944         | 5,448 (42.09%)  |
| LibreOffice           | 11,982         | 3,975 (33.17%)  |
| Samba                 | 4,507          | 1,700 (37.72%)  |
| VTK                   | 2,522          | 1,166 (46.23%)  |
| OpenCV                | 2,443          | 710 (29.06%)    |
| Kodi                  | 2,393          | 682 (28.5%)     |
| !CHAOS Control System | 1,980          | 282 (14.24%)    |
| Chromium EC           | 964            | 306 (31.74%)    |
| Thunderbird           | 885            | 357 (40.34%)    |


In [35]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and language in ('C','C++')
and me.memory =1
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
eliminated=root[root['status']=='Fixed'].groupby('name')[['cid']].count()
eliminated.rename(columns={'cid':'e'},inplace=True)
bug=root[root['classification']=='Bug'].groupby('name')[['cid']].count()
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability ac on ac.alert_id=a.id
where is_invalid=0
and language in ('C','C++')
and actionability = 1
and me.memory =1
order by name;'''
df=pd.DataFrame(sql.execute(q))
actionable=df.groupby('name')[['cid']].count()
actionable.rename(columns={'cid':'a'},inplace=True)
bug.rename(columns={'cid':'b'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,eliminated, actionable, bug])
df.e = round((df.e/df.total)*100,2)    
df.a = round((df.a/df.total)*100,2)
df.b = round((df.b/df.total)*100,2)  
df=df.reset_index() 

In [36]:
df.e=df.e.astype(str)+'%'
df.a=df.a.astype(str)+'%'
df.b=df.b.astype(str)+'%'
df.rename(columns={'name':'Project','total':'Total Memory Alerts', 'e':'Eliminated Memory', 'a':'Actionable Memory','b':'Triaged Bug by Devs. Memory'}, inplace=True)
memory = df 
memory

,Project,Total Memory Alerts,Eliminated Memory,Actionable Memory,Triaged Bug by Devs. Memory
0,!CHAOS Control System,282,80.14%,39.01%,1.42%
1,Chromium EC,306,93.46%,29.74%,17.65%
2,Firefox,5448,77.22%,49.72%,12.43%
3,Kodi,682,68.62%,54.11%,25.07%
4,LibreOffice,3975,80.63%,73.46%,51.35%
5,Linux,7903,73.06%,43.54%,3.11%
6,OpenCV,710,89.15%,32.54%,0.28%
7,Samba,1700,77.88%,39.88%,3.82%
8,Thunderbird,357,63.31%,24.37%,12.61%
9,VTK,1166,12.78%,7.2%,0.43%


In [37]:
memory = memory.sort_values('Total Memory Alerts', ascending=False)
print(tabulate(memory, tablefmt='pipe',headers='keys',showindex=False))

| Project               |   Total Memory Alerts | Eliminated Memory   | Actionable Memory   | Triaged Bug by Devs. Memory   |
|:----------------------|----------------------:|:--------------------|:--------------------|:------------------------------|
| Linux                 |                  7903 | 73.06%              | 43.54%              | 3.11%                         |
| Firefox               |                  5448 | 77.22%              | 49.72%              | 12.43%                        |
| LibreOffice           |                  3975 | 80.63%              | 73.46%              | 51.35%                        |
| Samba                 |                  1700 | 77.88%              | 39.88%              | 3.82%                         |
| VTK                   |                  1166 | 12.78%              | 7.2%                | 0.43%                         |
| OpenCV                |                   710 | 89.15%              | 32.54%              | 0.28%                   

In [23]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and language in ('C','C++')
and me.memory is null
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
eliminated=root[root['status']=='Fixed'].groupby('name')[['cid']].count()
eliminated.rename(columns={'cid':'e'},inplace=True)
bug=root[root['classification']=='Bug'].groupby('name')[['cid']].count()
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability ac on ac.alert_id=a.id
where is_invalid=0
and language in ('C','C++')
and actionability = 1
and me.memory is null
order by name;'''
df=pd.DataFrame(sql.execute(q))
actionable=df.groupby('name')[['cid']].count()
actionable.rename(columns={'cid':'a'},inplace=True)
bug.rename(columns={'cid':'b'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,eliminated, actionable, bug])
df=df.reset_index()
df.e = round((df.e/df.total)*100,2)    
df.a = round((df.a/df.total)*100,2)
df.b = round((df.b/df.total)*100,2)   
# df.e=df.e.astype(str)+'%'
# df.a=df.a.astype(str)+'%'
# df.b=df.b.astype(str)+'%'
df.rename(columns={'name':'Project','total':'Total Non-Memory Alerts', 'e':'Eliminated Non-memory', 'a':'Actionable Non-memory','b':'Triaged Bugs by Devs. Non-memory'}, inplace=True)
non_memory = df        
#non_memory = non_memory.sort_values('total non_memory', ascending=False)
non_memory

,Project,Total Non-Memory Alerts,Eliminated Non-memory,Actionable Non-memory,Triaged Bugs by Devs. Non-memory
0,Chromium EC,658,71.6,28.0,8.5
1,Firefox,7496,70.9,46.5,5.1
2,Kodi,1711,67.3,50.3,11.6
3,LibreOffice,8007,94.1,70.2,39.8
4,Linux,11611,65.2,39.1,3.4
5,OpenCV,1733,76.5,36.0,0.4
6,Samba,2807,73.3,24.7,1.5
7,Thunderbird,528,49.8,15.0,5.1
8,VTK,1356,19.1,7.6,1.8


In [24]:
df=reduce(lambda x,y: pd.merge(x,y,on='Project'),[memory,non_memory])
df = df.sort_values('Total Memory Alerts', ascending=False)
df

,Project,Total Memory Alerts,Eliminated Memory,Actionable Memory,Triaged Bug by Devs. Memory,Total Non-Memory Alerts,Eliminated Non-memory,Actionable Non-memory,Triaged Bugs by Devs. Non-memory
0,Linux,7903,73.1,43.5,3.1,11611,65.2,39.1,3.4
1,Firefox,5448,77.2,49.7,12.4,7496,70.9,46.5,5.1
2,LibreOffice,3975,80.6,73.5,51.4,8007,94.1,70.2,39.8
3,Samba,1700,77.9,39.9,3.8,2807,73.3,24.7,1.5
4,VTK,1166,12.8,7.2,0.4,1356,19.1,7.6,1.8
5,OpenCV,710,89.2,32.5,0.3,1733,76.5,36.0,0.4
6,Kodi,682,68.6,54.1,25.1,1711,67.3,50.3,11.6
7,Thunderbird,357,63.3,24.4,12.6,528,49.8,15.0,5.1
8,Chromium EC,306,93.5,29.7,17.6,658,71.6,28.0,8.5


In [25]:
from scipy.stats import shapiro
stat, p = shapiro(df['Eliminated Memory'])
for col in df.columns:
    if col=='Project':
        continue
    stat, p = shapiro(df[col])
    print(col, stat, p)

Total Memory Alerts 0.808286190032959 0.025381118059158325
Eliminated Memory 0.7710899114608765 0.009508478455245495
Actionable Memory 0.9915132522583008 0.997808039188385
Triaged Bug by Devs. Memory 0.8192175030708313 0.033763788640499115
Total Non-Memory Alerts 0.8159152269363403 0.030980780720710754
Eliminated Non-memory 0.866422176361084 0.1125360056757927
Actionable Non-memory 0.9837080836296082 0.9806068539619446
Triaged Bugs by Devs. Non-memory 0.6523598432540894 0.0003933573898393661


In [26]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['Eliminated Memory'],df['Eliminated Non-memory']
)
stat, p

(29.0, 0.1656935481238075)

In [27]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['Actionable Memory'],df['Actionable Non-memory']
)
stat, p

(35.0, 0.3294216274746403)

In [28]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['Triaged Bug by Devs. Memory'],df['Triaged Bugs by Devs. Non-memory']
)
stat, p

(32.0, 0.23996434607297862)

In [29]:
df['Eliminated Memory'].median()

77.22

In [30]:
df['Actionable Memory'].median()

39.88

In [31]:
df['Triaged Bug by Devs. Memory'].median()

12.43